# Culinária ao Redor do Mundo: Análise de Receitas

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

Criação de Tabela

In [2]:
DROP TABLE IF EXISTS Receitas;

CREATE TABLE Receitas (
  id INTEGER NOT NULL,
  titulo VARCHAR(200),
  regiao VARCHAR(100),
  PRIMARY KEY(id)
) AS SELECT
  id, titulo, regiao
FROM CSVREAD('../data/processed/receitas.csv');


In [3]:
DROP TABLE IF EXISTS Nutrientes;

CREATE TABLE Nutrientes (
    id INTEGER NOT NULL,
    nome VARCHAR(50),
    PRIMARY KEY(id)
) AS SELECT
    id, nome
FROM CSVREAD('../data/processed/nutrientes.csv');

In [4]:
DROP TABLE IF EXISTS Compostos;

CREATE TABLE Compostos (
    id INTEGER NOT NULL,
    nome VARCHAR(300),
    PRIMARY KEY(id)
) AS SELECT
    id, nome
FROM CSVREAD('../data/processed/compostos.csv');

In [5]:
DROP TABLE IF EXISTS Ingredientes;

CREATE TABLE Ingredientes (
    id INTEGER NOT NULL,
    nome VARCHAR(300),
    grupo VARCHAR(300),
    subgrupo VARCHAR(300),
    densidade FLOAT,
    peso_medio FLOAT,
    composto BIT,
    PRIMARY KEY(id)
) AS SELECT
    id, nome, grupo, subgrupo, densidade, peso_medio,composto
FROM CSVREAD('../data/processed/ingredientes.csv');

In [6]:
DROP TABLE IF EXISTS ingredientes_receitas;

CREATE TABLE Ingredientes_receitas (
    id_ingrediente INTEGER NOT NULL,
    id_receita INTEGER NOT NULL,
    volume FLOAT NOT NULL,
    massa FLOAT NOT NULL,
    unidade INTEGER NOT NULL,
    PRIMARY KEY (id_ingrediente, id_receita)
) AS SELECT
    id_ingrediente, id_receita, volume, massa, unidade
FROM CSVREAD('../data/processed/ingredientes_receitas.csv');

In [7]:
DROP TABLE IF EXISTS Composicao_ingrediente;

CREATE TABLE Composicao_ingrediente (
  id_composto INTEGER NOT NULL,
  id_base INTEGER NOT NULL,
  PRIMARY KEY (id_composto, id_base)
) AS SELECT
    id_composto, id_base
FROM CSVREAD('../data/processed/composicao.csv')

In [8]:
DROP TABLE IF EXISTS Compostos_content;

CREATE TABLE Compostos_content (
  id_ingrediente INTEGER NOT NULL,
  id_composto INTEGER NOT NULL,
  quantidade FLOAT NOT NULL,
  PRIMARY KEY (id_ingrediente, id_composto)
) AS SELECT
    id_ingrediente, id_composto, quantidade
FROM CSVREAD('../data/processed/content_compostos.csv')

In [9]:
DROP TABLE IF EXISTS Nutrientes_content;

CREATE TABLE Nutrientes_content (
  id_ingrediente INTEGER NOT NULL,
  id_nutriente INTEGER NOT NULL,
  quantidade FLOAT NOT NULL,
  PRIMARY KEY (id_ingrediente, id_nutriente)
) AS SELECT
    id_ingrediente, id_nutriente, quantidade
FROM CSVREAD('../data/processed/content_nutrientes.csv')

1)Quais as estruturas bioquímicas mais frequentes em cada região?

In [10]:
DROP VIEW IF EXISTS EstruturasFrequentes;
CREATE VIEW EstruturasFrequentes AS
SELECT 
    r.regiao AS Regiao,
    c.nome AS NomeComposto,
    COUNT(*) AS Quantidade
FROM 
    Receitas r
JOIN 
    Ingredientes_receitas ir ON r.id = ir.id_receita
JOIN 
    Compostos_content cc ON ir.id_ingrediente = cc.id_ingrediente
JOIN 
    Compostos c ON cc.id_composto = c.id
GROUP BY 
    r.regiao, c.nome
ORDER BY 
    COUNT(*) DESC;
    
SELECT r.regiao, (SELECT COUNT(*) FROM EstruturasFrequentes ef WHERE ef.regiao = r.regiao ORDER BY COUNT(*) DESC LIMIT 1) FROM regioes r;

org.h2.jdbc.JdbcSQLException:  Table "REGIOES" not found; SQL statement

2)Quais regiões possuem receitas com ingredientes majoritariamente vegetais?

In [11]:
DROP VIEW IF EXISTS RegionVegetablePercentageRanking;
CREATE VIEW RegionVegetablePercentageRanking AS
SELECT
    r.regiao AS nome_regiao,
    COUNT(DISTINCT CASE WHEN i.grupo NOT IN ('Milk and milk products', 'Snack foods', 'Aquatic foods', 'Animal foods') THEN ir.id_ingrediente END) AS total_vegetais,
    COUNT(DISTINCT ir.id_ingrediente) AS total_ingredientes,
    COUNT(DISTINCT ir.id_receita) AS total_receitas,
    (COUNT(DISTINCT CASE WHEN i.grupo NOT IN ('Milk and milk products', 'Snack foods', 'Aquatic foods', 'Animal foods') THEN ir.id_ingrediente END) * 100.0) / COUNT(DISTINCT ir.id_ingrediente) AS porcentagem_vegetais
FROM
    Receitas r
JOIN
    Ingredientes_receitas ir ON r.id = ir.id_receita
JOIN
    Ingredientes i ON ir.id_ingrediente = i.id
GROUP BY
    r.regiao
ORDER BY
    porcentagem_vegetais DESC;
SELECT * FROM RegionVegetablePercentageRanking LIMIT 22;


3)Quais regiões possuem a maior média de gorduras por receita?

In [12]:
DROP VIEW IF EXISTS RegionFatAverageRanking;
CREATE VIEW RegionFatAverageRanking AS
SELECT
    r.regiao AS nome_regiao,
    AVG(nc.quantidade) AS media_gordura
FROM
    Receitas r
JOIN
    Ingredientes_receitas ir ON r.id = ir.id_receita
JOIN
    Nutrientes_content nc ON ir.id_ingrediente = nc.id_ingrediente
WHERE
    nc.id_nutriente = 1
GROUP BY
    r.regiao
ORDER BY media_gordura DESC;
SELECT * FROM RegionFatAverageRanking LIMIT 22;